In [1]:
with open('systemsproject.txt', 'r') as file:
    content = file.read()
    source_code = file.readlines()

print(content)                                                                            

SUM

Start
0000
I

LDX
#0

LDA
#0

+LDB
#TABLE2

BASE
TABLE2
LOOP

ADD
TABLE,X


ADD
TABLE2,X


TIX
COUNT


JLT
LOOP

SUM     	START		0000
		LDX		#0
		LDA 		#0
		+LDB 		#TABLE2
		BASE		TABLE2
LOOP		ADD		TABLE,X
		ADD		TABLE2,X
		TIX 		COUNT
		JLT		LOOP
		+STA		TOTAL
		RSUB 		
COUNT		RESW		1
TABLE  		RESW		2000
TABLE2		RESW		2000
TOTAL		RESW		1
		END 		SUM 


In [2]:
from IPython.display import display
from ipywidgets import FileUpload
import io


In [3]:
def on_upload_change(change):
    if uploader.value:
        uploaded_file = list(uploader.value.values())[0]
        content = io.StringIO(uploaded_file['content'].decode('utf-8'))
        source_code = content.readlines()

In [4]:
OPTAB = {
    'ADD':    {'opcode': '18', 'format': 3},
    'ADDF':   {'opcode': '58', 'format': 3},
    'ADDR':   {'opcode': '90', 'format': 2},
    'AND':    {'opcode': '40', 'format': 3},
    'CLEAR':  {'opcode': 'B4', 'format': 2},
    'COMP':   {'opcode': '28', 'format': 3},
    'COMPF':  {'opcode': '88', 'format': 3},
    'COMPR':  {'opcode': 'A0', 'format': 2},
    'DIV':    {'opcode': '24', 'format': 3},
    'DIVF':   {'opcode': '64', 'format': 3},
    'DIVR':   {'opcode': '9C', 'format': 2},
    'FIX':    {'opcode': 'C4', 'format': 1},
    'FLOAT':  {'opcode': 'C0', 'format': 1},
    'HIO':    {'opcode': 'F4', 'format': 1},
    'J':      {'opcode': '3C', 'format': 3},
    'JEQ':    {'opcode': '30', 'format': 3},
    'JGT':    {'opcode': '34', 'format': 3},
    'JLT':    {'opcode': '38', 'format': 3},
    'JSUB':   {'opcode': '48', 'format': 3},
    'LDA':    {'opcode': '00', 'format': 3},
    'LDB':    {'opcode': '68', 'format': 3},
    'LDCH':   {'opcode': '50', 'format': 3},
    'LDL':    {'opcode': '08', 'format': 3},
    'LDS':    {'opcode': '6C', 'format': 3},
    'LDT':    {'opcode': '74', 'format': 3},
    'LDX':    {'opcode': '04', 'format': 3},
    'MUL':    {'opcode': '20', 'format': 3},
    'MULR':   {'opcode': '98', 'format': 2},
    'OR':     {'opcode': '44', 'format': 3},
    'RD':     {'opcode': 'D8', 'format': 3},
    'RMO':    {'opcode': 'AC', 'format': 2},
    'RSUB':   {'opcode': '4C', 'format': 3},
    'SHIFTL': {'opcode': 'A4', 'format': 2},
    'SHIFTR': {'opcode': 'A8', 'format': 2},
    'STA':    {'opcode': '0C', 'format': 3},
    'STB':    {'opcode': '78', 'format': 3},
    'STCH':   {'opcode': '54', 'format': 3},
    'STL':    {'opcode': '14', 'format': 3},
    'STS':    {'opcode': '7C', 'format': 3},
    'STT':    {'opcode': '84', 'format': 3},
    'STX':    {'opcode': '10', 'format': 3},
    'SUB':    {'opcode': '1C', 'format': 3},
    'SUBR':   {'opcode': '94', 'format': 2},
    'SVC':    {'opcode': 'B0', 'format': 2},
    'TD':     {'opcode': 'E0', 'format': 3},
    'TIX':    {'opcode': '2C', 'format': 3},
    'TIXR':   {'opcode': 'B8', 'format': 2},
    'WD':     {'opcode': 'DC', 'format': 3}
}

In [5]:
def is_comment(line):
    return line.strip().startswith('.') or line.strip() == ''

def get_instruction_size(opcode, operand):
    if opcode.startswith('+'):
        return 4
    elif opcode in OPTAB:
        return OPTAB[opcode]["format"]
    elif opcode == "WORD":
        return 3
    elif opcode == "RESW":
        return 3 * int(operand)
    elif opcode == "RESB":
        return int(operand)
    elif opcode == "BYTE":
        if operand.startswith("C'"):
            return len(operand) - 3
        elif operand.startswith("X'"):
            return (len(operand) - 3) // 2
    elif opcode == "BASE" or opcode == "NOBASE":
        return 0
    return 0

In [6]:
def parse_line(line):
    parts = line.strip().split()
    if len(parts) == 3:
        return parts[0], parts[1], parts[2]
    elif len(parts) == 2:
        return '', parts[0], parts[1]
    elif len(parts) == 1:
        return '', parts[0], ''
    else:
        return '', '', ''

In [7]:
def pass1(source_lines):
    LOCCTR = 0
    SYMTAB = {}
    intermediate = []

    label, opcode, operand = parse_line(source_lines[0])
    if opcode == "START":
        LOCCTR = int(operand, 16)
        start_addr = LOCCTR
        intermediate.append((LOCCTR, source_lines[0]))
        line_index = 1
    else:
        LOCCTR = 0
        start_addr = 0
        line_index = 0

    for line in source_lines[line_index:]:
        if is_comment(line):
            intermediate.append(('', line))
            continue

        label, opcode, operand = parse_line(line)

        if label:
            if label in SYMTAB:
                raise ValueError(f"Duplicate symbol: {label}")
            SYMTAB[label] = LOCCTR

        instr_size = get_instruction_size(opcode, operand)
        intermediate.append((LOCCTR, line))
        LOCCTR += instr_size

        if opcode == "END":
            break

    program_length = LOCCTR - start_addr
    return SYMTAB, intermediate, program_length

In [8]:
source_code = [
    "SUM      START     0000",
    "         LDX       #0",
    "         LDA       #0",
    "         +LDB      #TABLE2",
    "         BASE      TABLE2",
    "LOOP     ADD       TABLE,X",
    "         ADD       TABLE2,X",
    "         TIX       COUNT",
    "         JLT       LOOP",
    "         +STA      TOTAL",
    "         RSUB",
    "COUNT    RESW      1",
    "TABLE    RESW      2000",
    "TABLE2   RESW      2000",
    "TOTAL    RESW      1",
    "         END       SUM"
]

In [9]:
if not source_code:
    print("Error: The file is empty or could not be read.")
else:
    SYMTAB, intermediate, prog_len = pass1(source_code)

In [10]:
print("Symbol Table (SYMTAB):")

for label, addr in SYMTAB.items():
    print(f"{label}: {hex(addr).upper()}")

print("\nIntermediate File:")
for addr, line in intermediate:
    if addr != '':
        print(f"{hex(addr).upper():<6} {line}")
    else:
        print(f"        {line}")

print(f"\nProgram Length: {hex(prog_len).upper()}")

Symbol Table (SYMTAB):
LOOP: 0XA
COUNT: 0X1D
TABLE: 0X20
TABLE2: 0X1790
TOTAL: 0X2F00

Intermediate File:
0X0    SUM      START     0000
0X0             LDX       #0
0X3             LDA       #0
0X6             +LDB      #TABLE2
0XA             BASE      TABLE2
0XA    LOOP     ADD       TABLE,X
0XD             ADD       TABLE2,X
0X10            TIX       COUNT
0X13            JLT       LOOP
0X16            +STA      TOTAL
0X1A            RSUB
0X1D   COUNT    RESW      1
0X20   TABLE    RESW      2000
0X1790 TABLE2   RESW      2000
0X2F00 TOTAL    RESW      1
0X2F03          END       SUM

Program Length: 0X2F03


In [11]:
with open("intermediate.txt", "w") as f:
    for addr, line in intermediate:
        if addr != '':
            f.write(f"{addr:04X}  {line.strip()}\n")
        else:
            f.write(f"      {line.strip()}\n")

print("Intermediate file (intermediate.txt) generated successfully.")

Intermediate file (intermediate.txt) generated successfully.


In [12]:
# Save SYMTAB to file
with open("symtab.txt", "w") as f:
    f.write("Symbol\tAddress\n")
    f.write("------------------\n")
    for symbol, address in SYMTAB.items():
        f.write(f"{symbol}\t{address:04X}\n")

print("Symbol Table (symtab.txt) generated successfully.")


Symbol Table (symtab.txt) generated successfully.


In [13]:
def pass2(intermediate, SYMTAB, OPTAB, program_name, start_addr):
    object_code_list = []  # Initialize the list to store object codes
    base = None

    for addr, line in intermediate:
        if line.strip() == '' or line.strip().startswith('.'):
            object_code_list.append((addr, line.strip(), ''))  # comment or blank line
            continue

        label, opcode, operand = parse_line(line)

        if opcode == 'START' or opcode == 'END':
            object_code_list.append((addr, line.strip(), ''))  # directives - no object code
            continue

        if opcode == 'BASE':
            base = SYMTAB.get(operand, None)
            object_code_list.append((addr, line.strip(), ''))  # BASE is a directive
            continue

        if opcode in ['RESW', 'RESB']:
            object_code_list.append((addr, line.strip(), ''))  # reservation - no object code
            continue

        is_format4 = opcode.startswith('+')
        pure_opcode = opcode[1:] if is_format4 else opcode

        if pure_opcode == 'RSUB':
            objcode = '4F0000'
            object_code_list.append((addr, line.strip(), objcode))
            continue

        opinfo = OPTAB.get(pure_opcode)
        if not opinfo:
            object_code_list.append((addr, line.strip(), ''))  # unknown opcode
            continue

        ophex = int(opinfo['opcode'], 16)
        n, i, x, b, p, e = 1, 1, 0, 0, 0, 0

        # Handle addressing mode
        if operand.startswith('#'):
            n, i = 0, 1
            operand = operand[1:]
        elif operand.startswith('@'):
            n, i = 1, 0
            operand = operand[1:]
        else:
            n, i = 1, 1

        # Handle indexed addressing
        if ',' in operand:
            symbol, index = operand.split(',')
            if index.upper() == 'X':
                x = 1
            operand = symbol

        if is_format4:
            e = 1
            disp = SYMTAB.get(operand, 0) if operand in SYMTAB else int(operand)
            address = disp
            nixbpe = (n << 5) + (i << 4) + (x << 3) + (b << 2) + (p << 1) + e
            objcode = ((ophex >> 2) << 6) + nixbpe
            full = (objcode << 20) + address
            object_code_list.append((addr, line.strip(), f"{full:08X}"))
        else:
            e = 0
            target_addr = SYMTAB.get(operand, None)

            if target_addr is None and operand.isdigit():
                disp = int(operand)
            elif target_addr is not None:
                pc = addr + 3
                disp = target_addr - pc
                if -2048 <= disp <= 2047:
                    p = 1
                elif base is not None:
                    disp = target_addr - base
                    b = 1
                else:
                    disp = 0  # fallback
            else:
                disp = 0  # fallback

            nixbpe = (n << 5) + (i << 4) + (x << 3) + (b << 2) + (p << 1) + e
            objcode = ((ophex >> 2) << 6) + nixbpe
            full = (objcode << 12) + (disp & 0xFFF)
            object_code_list.append((addr, line.strip(), f"{full:06X}"))

    return object_code_list



In [14]:
program_name = "SUM"
start_addr = 0x0000

obj_list = pass2(intermediate, SYMTAB, OPTAB, program_name, start_addr)

print("=== Object Code Output ===")
for addr, line, obj in obj_list:
    print(f"{hex(addr).upper():<6} {line:<35} {obj}")

=== Object Code Output ===
0X0    SUM      START     0000             
0X0    LDX       #0                        050000
0X3    LDA       #0                        010000
0X6    +LDB      #TABLE2                   69101790
0XA    BASE      TABLE2                    
0XA    LOOP     ADD       TABLE,X          1BA013
0XD    ADD       TABLE2,X                  1BC000
0X10   TIX       COUNT                     2F200A
0X13   JLT       LOOP                      3B2FF4
0X16   +STA      TOTAL                     0F102F00
0X1A   RSUB                                4F0000
0X1D   COUNT    RESW      1                
0X20   TABLE    RESW      2000             
0X1790 TABLE2   RESW      2000             
0X2F00 TOTAL    RESW      1                
0X2F03 END       SUM                       


In [15]:
obj_list = pass2(intermediate, SYMTAB, OPTAB, program_name, start_addr)

In [16]:
with open("out_pass2.txt", "w") as f:
    for addr, line, obj in obj_list:
        f.write(f"{addr:04X} {line:<35} {obj}\n")

In [17]:
def generate_HTME(program_name, start_addr, program_length, object_code_list):
    H_record = f"H^{program_name:<6}^{start_addr:06X}^{program_length:06X}"

    # Build T records (Text)
    T_records = []
    current_record = ""
    current_start = None
    current_length = 0

    for addr, line, obj in object_code_list:
        if obj == "":  # Skip lines without object code
            if current_record:
                T_records.append(f"T^{current_start:06X}^{current_length:02X}^{current_record}")
                current_record = ""
                current_length = 0
                current_start = None
            continue

        if current_start is None:
            current_start = addr

        if current_length + len(obj) // 2 > 30:
            T_records.append(f"T^{current_start:06X}^{current_length:02X}^{current_record}")
            current_record = obj
            current_start = addr
            current_length = len(obj) // 2
        else:
            current_record += f"^{obj}"
            current_length += len(obj) // 2

    if current_record:
        T_records.append(f"T^{current_start:06X}^{current_length:02X}^{current_record}")

    # M records (modification) — simplified: only for format 4 instructions (assumed address needs fixup)
    M_records = []
    for addr, line, obj in object_code_list:
        if line.startswith('+'):  # extended format instructions
            # Modify starting 5 half-bytes (20 bits), starting after 1.5 bytes = addr+1
            M_records.append(f"M^{addr+1:06X}^05")

    # End Record  
    E_record = f"E^{start_addr:06X}"

    # Combine all records
    with open("HTME.txt","w") as f:     
        f.write(H_record + "\n")
        for t in T_records:
            f.write(t + "\n")
        for m in M_records:
            f.write(m + "\n")        
        f.write(E_record + "\n")

    print("HTME.txt generated successfully.")  


In [18]:
generate_HTME(program_name, start_addr, prog_len, obj_list)    


HTME.txt generated successfully.
